In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
data_X = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
#Data parameters
image_size = 28*28
n_class = 10

#Training parameters
n_cluster = 25
total_epochs = 50

X = tf.placeholder(tf.float32, shape=[None, image_size], name='InputImage')
Y = tf.placeholder(tf.float32, shape=[None, n_class], name="Label")
with tf.name_scope("Kmeans"):
    kmeans = KMeans(inputs=X, num_clusters=n_cluster, distance_metric='squared_euclidean', use_mini_batch=True)
    all_scores, cluster_idx, scores, cluster_centers_initialized, init_op, train_op = kmeans.training_graph()
    cluster_idx = cluster_idx[0]
    avg_distance = tf.reduce_mean(scores)

init = tf.global_variables_initializer()

In [4]:
with tf.Session() as sess:
    sess.run(init)
    sess.run(init_op, feed_dict={X:data_X})
    
    for i in range(total_epochs):
        _, dis, idx = sess.run([train_op, avg_distance, cluster_idx], feed_dict={X: data_X})
    
    counts = np.zeros(shape=(n_cluster, n_class))
    for i in range(len(idx)):
        counts[idx[i]] += mnist.train.labels[i]
    labels_map = [np.argmax(c) for c in counts]
    labels_map = tf.convert_to_tensor(labels_map)

    # Evaluation ops
    # Lookup: centroid_id -> label
    cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
    # Compute accuracy
    correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Test Model
    test_x, test_y = mnist.test.images, mnist.test.labels
    print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Test Accuracy: 0.6963
